In [4]:
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from typing import Optional, Callable
import ipywidgets as wg
from fancy_einsum import einsum

import utils

# Fourier Transform


## Discrete Fourier Transform

(forgot that the outer product existed. whoops)

In [5]:
def DFT_1d(arr : np.ndarray) -> np.ndarray:
    """
    Returns the Discrete Fourier Transform of the array 'arr'
    """
    leftArray =  np.fromfunction(lambda x, y: np.power(np.power(np.e,-2 * np.pi * 1j / len(arr)),x * y), (arr.shape[0], arr.shape[0]), dtype=float)
    # print(leftArray.shape)
    # print(arr.shape)
    return leftArray@arr

DFT_1d(np.array([3,4,5]))
# expected: [12. +0.j        -1.5+0.8660254j -1.5-0.8660254j]

array([12. +0.j       , -1.5+0.8660254j, -1.5-0.8660254j])

### Inverse Discrete Fourier Transform

In [6]:
def DFT_1d(arr: np.ndarray, inverse: bool = False) -> np.ndarray:
    """
    Returns the DFT of the array `arr`, with an optional `inverse` argument.
    """
    if inverse:
        leftArray =  np.fromfunction(lambda x, y: np.power(np.power(np.e,2 * np.pi * 1j / len(arr)),x * y), (arr.shape[0], arr.shape[0]), dtype=float)
        return leftArray@arr / len(arr)
    else:
        leftArray =  np.fromfunction(lambda x, y: np.power(np.power(np.e,-2 * np.pi * 1j / len(arr)),x * y), (arr.shape[0], arr.shape[0]), dtype=float)
        return leftArray@arr
    

utils.test_DFT_func(DFT_1d)

### A sample test function 

(or, more accurately, lines of code)

In [7]:
x = np.array([1, 2 - 1j, -1j, -1 + 2j])
expectedTransform = (2, -2 - 2j, -2j, 4 + 4j)
gotTransform = DFT_1d(x)
np.testing.assert_allclose(expectedTransform,gotTransform, atol=1e-10)

## Continuous Fourier Transform

### Helper Methods

In [8]:
def integrate_function(func: Callable, x0: float, x1: float, n_samples: int = 1000):
    """
    Calculates the approximation of the Riemann integral of the function `func`, 
    between the limits x0 and x1.

    You should use the Left Rectangular Approximation Method (LRAM).
    """

    x_width = (x1 - x0) / n_samples
    total = 0
    for i in range(n_samples):
        total += func(i * x_width + x0) * x_width

    return total

utils.test_integrate_function(integrate_function)

In [9]:
def integrate_product(func1: Callable, func2: Callable, x0: float, x1: float):
    """
    Computes the integral of the function x -> func1(x) * func2(x).
    """

    def product(x):
        return func1(x) * func2(x)

    return integrate_function(product,x0,x1)

utils.test_integrate_product(integrate_product)

### Fourier Series

In [10]:
def calculate_fourier_series(func: Callable, max_freq: int = 50):
    """
    Calculates the fourier coefficients of a function, 
    assumed periodic between [-pi, pi].

    Your function should return ((a_0, A_n, B_n), func_approx), where:
        a_0 is a float
        A_n, B_n are lists of floats, with n going up to `max_freq`
        func_approx is the fourier approximation, as described above
    """
    a0 = 1 / np.pi * integrate_function(func,-1 * np.pi, np.pi)
    A_n = []
    B_n = []

    def modifiedCos(x, i):
        return np.cos(x * i)

    for i in range(1,50):
        print(i)
        newFunc = lambda x: np.cos(i*x)
        A_n.append(1 / np.pi * integrate_product(func, newFunc, -1 * np.pi, np.pi))
        newFunc = lambda y: np.sin(i*x)
        B_n.append(1 / np.pi * integrate_product(func, newFunc, -1 * np.pi, np.pi))
    
    def newFunction(x):
        output = a0 / 2
        for i in range(1,50):
            output += A_n[i] * np.cos( (i-0) * x)
            output += B_n[i] * np.sin( (i-0) * x)
        return output
    return ((a0, A_n, B_n), newFunction)

step_func = lambda x: 1 * (x > 0)
create_interactive_fourier_graph(calculate_fourier_series, func = step_func)

NameError: name 'create_interactive_fourier_graph' is not defined

In [11]:
wg._version

<module 'ipywidgets._version' from '/Users/codyrushing/mambaforge/envs/ARENAenv/lib/python3.9/site-packages/ipywidgets/_version.py'>

# Basic Neural Network


*okay, this one was rough. I spent over an hour on this section jsut trying to work it out, and I enventually had to look up the answers for some guidance. I have some of the high-level ideas down, but there are some small parts to this that I need to flush out.*

In [54]:
import pdb

NUM_FREQUENCIES = 10
TARGET_FUNC = lambda x: 1 * (x > 1)
TOTAL_STEPS = 4000
LEARNING_RATE = 1e-6

x = np.linspace(-np.pi, np.pi, 2000)
y = TARGET_FUNC(x)

x_cos = np.array([np.cos(n*x) for n in range(1, NUM_FREQUENCIES+1)])
x_sin = np.array([np.sin(n*x) for n in range(1, NUM_FREQUENCIES+1)])


a_0 = np.random.randn()
A_n = np.random.randn(NUM_FREQUENCIES)
B_n = np.random.randn(NUM_FREQUENCIES)

y_pred_list = []
coeffs_list = []

for step in range(TOTAL_STEPS):

    y_pred = 0.5 * a_0 + x_cos.T @ A_n + x_sin.T @ B_n

    lossArray:np.ndarray = (y_pred - y)
    loss = np.square(lossArray).sum()

    if step % 100 == 0:
        print(f"loss = {loss:.2f}")
        coeffs_list.append([a_0, A_n.copy(), B_n.copy()])
        y_pred_list.append(y_pred)

    dL_dy = 2.0 * (y_pred - y)
    dL_da0 = 0.5 * dL_dy.sum()
    dL_da = x_cos @ dL_dy
    dL_db = x_sin @ dL_dy
    # pdb.set_trace() 


    a_0 -= LEARNING_RATE * dL_da0 
    A_n -= LEARNING_RATE * dL_da
    B_n -= LEARNING_RATE * dL_db
    #pdb.set_trace() 


utils.visualise_fourier_coeff_convergence(x, y, y_pred_list, coeffs_list)

loss = 21682.73
loss = 14546.42
loss = 9762.67
loss = 6555.61
loss = 4405.31
loss = 2963.34
loss = 1996.21
loss = 1347.40
loss = 912.02
loss = 619.77
loss = 423.52
loss = 291.67
loss = 203.03
loss = 143.41
loss = 103.26
loss = 76.20
loss = 57.94
loss = 45.60
loss = 37.24
loss = 31.57
loss = 27.71
loss = 25.08
loss = 23.27
loss = 22.03
loss = 21.18
loss = 20.58
loss = 20.16
loss = 19.87
loss = 19.66
loss = 19.51
loss = 19.40
loss = 19.32
loss = 19.27
loss = 19.22
loss = 19.19
loss = 19.17
loss = 19.15
loss = 19.13
loss = 19.12
loss = 19.11


## PyTorch and Tensors

In [55]:
# "the constructor way is fraught with peril"

import torch
x = torch.arange(5)
y1 = torch.Tensor(x.shape)
y2 = torch.Tensor(tuple(x.shape))
y3 = torch.Tensor(list(x.shape))
print(y1, y2, y3)

tensor([2.8131e+20, 1.7566e+25, 1.7748e+28, 0.0000e+00, 0.0000e+00]) tensor([5.]) tensor([5.])


In [58]:
# torch.tensor([1,2,3,4]).mean() will not run!

In [59]:
import torch as t

In [73]:
from bdb import set_trace


NUM_FREQUENCIES = 2
TARGET_FUNC = lambda x: 1 * (x > 1)
TOTAL_STEPS = 4000
LEARNING_RATE = 1e-6

x = t.linspace(-t.pi, t.pi, 2000)
y = TARGET_FUNC(x)

x_cos = t.stack([t.cos(n*x) for n in range(1, NUM_FREQUENCIES+1)])
x_sin = t.stack([t.sin(n*x) for n in range(1, NUM_FREQUENCIES+1)])

a_0 = t.rand(1)
A_n = t.rand(NUM_FREQUENCIES)
B_n = t.rand(NUM_FREQUENCIES)

y_pred_list = []
coeffs_list = []

for step in range(TOTAL_STEPS):

    y_pred = 0.5 * a_0 + x_cos.T @ A_n + x_sin.T @ B_n

    lossArray = (y_pred - y)
    loss = t.square(lossArray).sum()

    if step % 100 == 0:
        print(f"loss = {loss:.2f}")
        # don't quite get where the to("cpu") in the solution comes from, but here we go
        coeffs_list.append([a_0.item(), A_n.to("cpu").numpy().copy(), B_n.to("cpu").numpy().copy()])
        y_pred_list.append(y_pred.numpy().copy())

    dL_dy = 2.0 * (y_pred - y)
    dL_da0 = 0.5 * dL_dy.sum()
    dL_da = x_cos @ dL_dy
    dL_db = x_sin @ dL_dy


    a_0 -= LEARNING_RATE * dL_da0 
    A_n -= LEARNING_RATE * dL_da
    B_n -= LEARNING_RATE * dL_db
    #pdb.set_trace() 


utils.visualise_fourier_coeff_convergence(x.numpy().copy(), y.numpy().copy(), y_pred_list, coeffs_list)

loss = 1703.44
loss = 1181.91
loss = 829.02
loss = 589.76
loss = 427.15
loss = 316.31
loss = 240.51
loss = 188.47
loss = 152.58
loss = 127.69
loss = 110.32
loss = 98.13
loss = 89.50
loss = 83.34
loss = 78.91
loss = 75.68
loss = 73.32
loss = 71.56
loss = 70.25
loss = 69.26
loss = 68.50
loss = 67.92
loss = 67.46
loss = 67.11
loss = 66.83
loss = 66.61
loss = 66.43
loss = 66.29
loss = 66.18
loss = 66.09
loss = 66.02
loss = 65.96
loss = 65.91
loss = 65.87
loss = 65.84
loss = 65.81
loss = 65.79
loss = 65.77
loss = 65.76
loss = 65.75


## Autograd

In [74]:
a = t.tensor(2, dtype = torch.float, requires_grad=True)
Q = 3 * a


tensor(2., requires_grad=True)


In [79]:
from bdb import set_trace
from pydoc import doc


NUM_FREQUENCIES = 2
TARGET_FUNC = lambda x: 1 * (x > 1)
TOTAL_STEPS = 4000
LEARNING_RATE = 1e-6

x = t.linspace(-t.pi, t.pi, 2000)
y = TARGET_FUNC(x)

x_cos = t.stack([t.cos(n*x) for n in range(1, NUM_FREQUENCIES+1)])
x_sin = t.stack([t.sin(n*x) for n in range(1, NUM_FREQUENCIES+1)])

a_0 = t.rand(1, dtype=torch.float, requires_grad=True)
A_n = t.rand(NUM_FREQUENCIES, dtype=torch.float, requires_grad=True)
B_n = t.rand(NUM_FREQUENCIES, dtype=torch.float, requires_grad=True)

y_pred_list = []
coeffs_list = []

for step in range(TOTAL_STEPS):

    y_pred = 0.5 * a_0 + x_cos.T @ A_n + x_sin.T @ B_n

    lossArray = (y_pred - y)
    loss = t.square(lossArray).sum()

    if step % 100 == 0:
        print(f"loss = {loss:.2f}")
        # don't quite get where the to("cpu") in the solution comes from, but here we go
        coeffs_list.append([a_0.item(), A_n.detach().to("cpu").numpy().copy(), B_n.detach().to("cpu").numpy().copy()])
        y_pred_list.append(y_pred.detach().numpy().copy())


    loss.backward()

    with torch.no_grad():
        a_0 -= LEARNING_RATE * a_0.grad # compiler is throwing an error here. dunno why!
        A_n -= LEARNING_RATE * A_n.grad
        B_n -= LEARNING_RATE * B_n.grad
        a_0.grad = None
        A_n.grad = None
        B_n.grad = None
        
    #pdb.set_trace() 


utils.visualise_fourier_coeff_convergence(x.numpy().copy(), y.numpy().copy(), y_pred_list, coeffs_list)

loss = 1499.13
loss = 1026.22
loss = 709.34
loss = 497.01
loss = 354.72
loss = 259.38
loss = 195.49
loss = 152.68
loss = 123.99
loss = 104.76
loss = 91.88
loss = 83.25
loss = 77.46
loss = 73.58
loss = 70.98
loss = 69.24
loss = 68.07
loss = 67.29
loss = 66.77
loss = 66.41
loss = 66.18
loss = 66.02
loss = 65.91
loss = 65.84
loss = 65.79
loss = 65.76
loss = 65.74
loss = 65.73
loss = 65.72
loss = 65.71
loss = 65.71
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70


## Models

In [90]:
from pdb import set_trace


NUM_FREQUENCIES = 2
TARGET_FUNC = lambda x: 1 * (x > 1)
TOTAL_STEPS = 4000
LEARNING_RATE = 1e-6

x = t.linspace(-t.pi, t.pi, 2000, dtype = t.float)
y = TARGET_FUNC(x)

x_cos = t.stack([t.cos(n*x) for n in range(1, NUM_FREQUENCIES+1)])
x_sin = t.stack([t.sin(n*x) for n in range(1, NUM_FREQUENCIES+1)])

x_input = t.concat([x_cos, x_sin], dim = 0).T
model = torch.nn.Sequential(torch.nn.Linear(2 * NUM_FREQUENCIES, 1), torch.nn.Flatten(0, 1))

# a_0 = t.rand(1, dtype=torch.float, requires_grad=True)
# A_n = t.rand(NUM_FREQUENCIES, dtype=torch.float, requires_grad=True)
# B_n = t.rand(NUM_FREQUENCIES, dtype=torch.float, requires_grad=True)

y_pred_list = []
coeffs_list = []

for step in range(TOTAL_STEPS):

    y_pred = model(x_input)

    lossArray = (y_pred - y)
    loss = t.square(lossArray).sum()

    # copied :)
    if step % 100 == 0:
        print(f"{loss = :.2f}")
        A_n = list(model.parameters())[0].detach().numpy()[:3].squeeze()
        B_n = list(model.parameters())[0].detach().numpy()[:6].squeeze()
        a_0 = list(model.parameters())[1].item()
        y_pred_list.append(y_pred.cpu().detach().numpy())
        coeffs_list.append([a_0, A_n.copy(), B_n.copy()])


    loss.backward()
    with torch.no_grad():
        for param in model.parameters():
            param -= LEARNING_RATE * param.grad
    model.zero_grad()
    #pdb.set_trace() 


utils.visualise_fourier_coeff_convergence(x.numpy().copy(), y.numpy().copy(), y_pred_list, coeffs_list)

loss = 1931.37
loss = 1221.65
loss = 798.00
loss = 537.42
loss = 373.26
loss = 267.96
loss = 199.50
loss = 154.58
loss = 124.91
loss = 105.21
loss = 92.10
loss = 83.36
loss = 77.52
loss = 73.61
loss = 71.00
loss = 69.25
loss = 68.07
loss = 67.29
loss = 66.76
loss = 66.41
loss = 66.17
loss = 66.02
loss = 65.91
loss = 65.84
loss = 65.79
loss = 65.76
loss = 65.74
loss = 65.72
loss = 65.72
loss = 65.71
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70


## Optimizers

In [92]:
from pdb import set_trace


NUM_FREQUENCIES = 2
TARGET_FUNC = lambda x: 1 * (x > 1)
TOTAL_STEPS = 4000
LEARNING_RATE = 1e-6

x = t.linspace(-t.pi, t.pi, 2000, dtype = t.float)
y = TARGET_FUNC(x)

x_cos = t.stack([t.cos(n*x) for n in range(1, NUM_FREQUENCIES+1)])
x_sin = t.stack([t.sin(n*x) for n in range(1, NUM_FREQUENCIES+1)])

x_input = t.concat([x_cos, x_sin], dim = 0).T
model = torch.nn.Sequential(torch.nn.Linear(2 * NUM_FREQUENCIES, 1), torch.nn.Flatten(0, 1))
optimizer = t.optim.Adam(model.parameters(), lr=0.001)
# a_0 = t.rand(1, dtype=torch.float, requires_grad=True)
# A_n = t.rand(NUM_FREQUENCIES, dtype=torch.float, requires_grad=True)
# B_n = t.rand(NUM_FREQUENCIES, dtype=torch.float, requires_grad=True)

y_pred_list = []
coeffs_list = []

for step in range(TOTAL_STEPS):

    y_pred = model(x_input)

    lossArray = (y_pred - y)
    loss = t.square(lossArray).sum()

    # copied :)
    if step % 100 == 0:
        print(f"{loss = :.2f}")
        A_n = list(model.parameters())[0].detach().numpy()[:3].squeeze()
        B_n = list(model.parameters())[0].detach().numpy()[:6].squeeze()
        a_0 = list(model.parameters())[1].item()
        y_pred_list.append(y_pred.cpu().detach().numpy())
        coeffs_list.append([a_0, A_n.copy(), B_n.copy()])


    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    #pdb.set_trace() 


utils.visualise_fourier_coeff_convergence(x.numpy().copy(), y.numpy().copy(), y_pred_list, coeffs_list)

loss = 889.94
loss = 609.66
loss = 429.69
loss = 307.42
loss = 221.89
loss = 163.29
loss = 124.55
loss = 99.91
loss = 84.84
loss = 75.99
loss = 71.00
loss = 68.31
loss = 66.92
loss = 66.24
loss = 65.93
loss = 65.79
loss = 65.73
loss = 65.71
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
loss = 65.70
